# 2D Visualization of 3D Data
## Zachary Neronha, Wong Lab
### 26 October 2018, created for the purpose of simple viewing of data in two dimensions 

In [1]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as cm
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib inline
from __future__ import division
init_notebook_mode(connected=True)
import plotly.plotly as py
import plotly
plotly.tools.set_credentials_file(username='zjneronha', api_key='PMrKGUGAet6kUZ5Rcf5E')

In [10]:
#import data from csv
xlsZ = pd.ExcelFile("NuclearTracking/ConvertedData/w18_combineddata.xls")
dxx = pd.read_excel(xlsZ,'xStore')
dyy = pd.read_excel(xlsZ,'yStore')
dzz = pd.read_excel(xlsZ,'zStore')

In [30]:
N = 500
random_x = np.linspace(-10,10,1001)
random_y = random_x**2

# Create a trace
trace = go.Scatter(
    x = random_x,
    y = random_y
    )

trace2 = go.Scatter(
    x = random_x,
    y = random_y*-1
    )

data = [trace,trace2]

py.iplot(data, filename='basic-line')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~zjneronha/0 or inside your plot.ly account where it is named 'basic-line'


In [ ]:
plt_data = []
trace = go.Scatter(
            name = 'SyntheticSphere',
            x = Positions[:,1],
            y = Positions[:,2],
            mode = 'markers',
            marker = dict(
                color = 1,
                size = 8,
                symbol = 'circle',
                line = dict(
                    color = 'rgb(180, 180, 180)',
                    width = 1.0
                ),
                opacity = 0.4
            )
        )

plt_data.append(trace)

layout = go.Layout(margin = dict(l = 0, r = 0, b = 0, t = 0))

fig = go.Figure(data=plt_data, layout=layout)

iplot(fig) 